In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib.pyplot as plt
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake.all
    from pydrake.all import (
        RigidTransform, RotationMatrix, RollPitchYaw, RevoluteSpring, LogOutput, SignalLogger, ForceElement, 
        DoorHinge, DoorHingeConfig, PidController, FixedInputPortValue
    )

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

In [ ]:
# Constants
in2m = 0.0254

PAPER_WIDTH = 11*in2m
PAPER_DEPTH = 8.5*in2m
PAPER_HEIGHT = 0.005
PAPER_DENSITY = 80/1000

PEDESTAL_WIDTH = 0.225
PEDESTAL_HEIGHT = 0.1

FINGER_RADIUS = 0.01

epsilon = 0.001

In [ ]:
def AddPedestal(plant):
    mu = 5.0
    # Parse pedestal model
    parser = pydrake.multibody.parsing.Parser(plant)
    pedestal_instance = parser.AddModelFromFile("pedestal.sdf")
    
    pedestal_body = plant.GetBodyByName("pedestal_base", pedestal_instance)
    width=PEDESTAL_WIDTH
    depth=0.2
    height=0.1
    i=0
    for x in [-width/2.0, width/2.0]:
        for y in [-depth/2.0, depth/2.0]:
            for z in [-height/2.0, height/2.0]:
                plant.RegisterCollisionGeometry(
                    pedestal_body,
                    RigidTransform([x, y, z]),
                    pydrake.geometry.Sphere(radius=1e-7), f"_contact_sphere{i}",
                    pydrake.multibody.plant.CoulombFriction(mu, mu))
                i += 1
    
    # Weld pedestal to world
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("pedestal_base", pedestal_instance),
        RigidTransform(RotationMatrix(), [0, 0, 0.1/2])
    )
    
    return pedestal_instance

In [ ]:
def AddPointFinger(plant):
    mu = 1.0
    radius = FINGER_RADIUS
    finger = plant.AddModelInstance("finger")

    # Add false body at the origin that the finger 
    false_body = plant.AddRigidBody("false_body", finger,
        pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))

    # Initialize finger body
    finger_body = plant.AddRigidBody("body", finger, 
        pydrake.multibody.tree.SpatialInertia(
            mass=1.0,
            p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=pydrake.multibody.tree.UnitInertia(1.0, 1.0, 1.0)))

    # Register geometry
    shape = pydrake.geometry.Sphere(radius)
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(
            finger_body, RigidTransform(), shape, "body", pydrake.multibody.plant.CoulombFriction(mu, mu))
        plant.RegisterVisualGeometry(finger_body, RigidTransform(), shape, "body", [.9, .5, .5, 1.0])
    
    # Add control joins for x and z movement
    finger_x = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint(
        "finger_x",
        plant.world_frame(),
        plant.GetFrameByName("false_body"), [1, 0, 0], -.3, .3))
    plant.AddJointActuator("finger_x", finger_x)
    finger_z = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint(
        "finger_z",
        plant.GetFrameByName("false_body"),
        plant.GetFrameByName("body"), [0, 0, 1], 0.0, 0.5))
    finger_z.set_default_translation(PEDESTAL_HEIGHT+radius+PAPER_HEIGHT+epsilon)
    plant.AddJointActuator("finger_z", finger_z)

    return finger

In [ ]:
class FingerControl(pydrake.systems.framework.LeafSystem):

    def __init__(self, plant):
        pydrake.systems.framework.LeafSystem.__init__(self)
        self._plant = plant

        self.DeclareVectorOutputPort(
            "finger_actuation", pydrake.systems.framework.BasicVector(2), 
                                    self.CalcOutput)

    def CalcOutput(self, context, output):
        finger_mass = 1
        g = self._plant.gravity_field().gravity_vector()[[0,2]]
        output.SetFromVector(-finger_mass*g)

In [ ]:
def AddPaper(plant, mass=None, mu=5.0, width=PAPER_WIDTH, depth=PAPER_DEPTH, height=PAPER_HEIGHT, name="paper", num_links=5, default_joint_angle=-np.pi/12):
    if num_links < 2:
        raise ValueError("num_links must be >= 2, but passed value is {}".format(num_links))
    
    if mass is None:
        mass = PAPER_DENSITY*width*height
        
    paper_instances = []
    link_width = width/num_links
    
    for link_num in range(num_links):
        paper_instance = plant.AddModelInstance(name + str(link_num))
        paper_body = plant.AddRigidBody(
            name + "_body" + str(link_num),
            paper_instance,
            pydrake.multibody.tree.SpatialInertia(mass=mass, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=pydrake.multibody.tree.UnitInertia.SolidBox(link_width, depth, height))
        )

        # Set up collision
        if plant.geometry_source_is_registered():
            plant.RegisterCollisionGeometry(
                paper_body,
                RigidTransform(),
                pydrake.geometry.Box(link_width, depth, height),
                name + "_body" + str(link_num), pydrake.multibody.plant.CoulombFriction(mu, mu)
            )
            i=0
            for x in [-link_width/2.0, link_width/2.0]:
                for y in [-depth/2.0, depth/2.0]:
                    for z in [-height/2.0, height/2.0]:
                        plant.RegisterCollisionGeometry(
                            paper_body,
                            RigidTransform([x, y, z]),
                            pydrake.geometry.Sphere(radius=1e-7), f"contact_sphere{i}_{link_num}",
                            pydrake.multibody.plant.CoulombFriction(mu, mu))
                        i += 1
            plant.RegisterVisualGeometry(
                paper_body,
                RigidTransform(),
                pydrake.geometry.Box(link_width, depth, height),
                name + "_body" + str(link_num),
                [0.9, 0.9, 0.9, 1.0])
        
        if link_num > 0:
            paper1_hinge_frame = pydrake.multibody.tree.FixedOffsetFrame(
                "paper_hinge_frame",
                plant.GetBodyByName("paper_body{}".format(link_num-1), paper_instances[-1]),
                RigidTransform(RotationMatrix(), [link_width/2+epsilon/2, 0, 0.5*height]))
            plant.AddFrame(paper1_hinge_frame)
            paper2_hinge_frame = pydrake.multibody.tree.FixedOffsetFrame(
                "paper_hinge_frame",
                plant.GetBodyByName("paper_body{}".format(link_num), paper_instance),
                RigidTransform(RotationMatrix(), [(-link_width/2+epsilon/2), 0, 0.5*height]))
            plant.AddFrame(paper2_hinge_frame)

            joint = plant.AddJoint(pydrake.multibody.tree.RevoluteJoint(
                "paper_hinge",
                paper1_hinge_frame,
                paper2_hinge_frame,
                [0, 1, 0]))

            joint.set_default_angle(default_joint_angle)
            ja = plant.AddJointActuator("joint actuator", joint)
                
        paper_instances.append(paper_instance)
    
    return paper_instances

In [ ]:
## Pre-finalize steps
builder = pydrake.systems.framework.DiagramBuilder()

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
pedestal_instance = AddPedestal(plant)
paper_instances = AddPaper(plant, num_links=10, default_joint_angle=-np.pi/60)
finger_instance = AddPointFinger(plant)
finger_ctrlr = FingerControl(plant)

# Fix paper to object
plant.WeldFrames(
    plant.world_frame(),
    plant.GetBodyByName("paper_body0", paper_instances[0]).body_frame(),
    RigidTransform(RotationMatrix(), [-(PEDESTAL_WIDTH/2-PAPER_WIDTH/4), 0, PEDESTAL_HEIGHT+PAPER_HEIGHT/2])
)

In [ ]:
plant.Finalize()

## Post finalize steps
paper_ctrlrs = []
for paper_instance in paper_instances[1:]:
    # Set up controllers for paper joints
    # In an ideal world, this would be done with a custom force element.
    # But since this is in Python, we can't implement the C++ class.
    paper_ctrlr = PidController(kp=[[0.0001]], ki=[[0]], kd=[[0.00001]])
    builder.AddSystem(paper_ctrlr)
    paper_ctrlrs.append(paper_ctrlr)
    builder.Connect(paper_ctrlr.get_output_port(), plant.get_actuation_input_port(paper_instance))
    builder.Connect(plant.get_state_output_port(paper_instance), paper_ctrlr.get_input_port_estimated_state())

# Conect finger controller
builder.AddSystem(finger_ctrlr)
builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))

# Visualization and logging
poses_logger = LogOutput(plant.get_state_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

# The paper controllers need to be fed an input state
for paper_ctrlr in paper_ctrlrs:
    paper_ctrlr_context = diagram.GetMutableSubsystemContext(paper_ctrlr, diagram_context)
    paper_ctrlr.get_input_port_desired_state().FixValue(paper_ctrlr_context, [0, 0])

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
simulator.AdvanceTo(5)
vis.stop_recording()
vis.publish_recording()

In [ ]:
%matplotlib notebook

plt.plot(poses_logger.sample_times(), poses_logger.data()[0,:], label="Position")
plt.plot(poses_logger.sample_times(), poses_logger.data()[1,:], label="Velocity")
plt.axhline(-np.pi/2, linestyle='--', color='k', label='Setpoint')
plt.legend()
plt.xlabel("Time (s)")
plt.show()